Cui Yulong [190198653]

I have modified my_utils.py file, adding a write() function to train_ch3() that creates a .txt file to record test accuracy. (See line 329 in my_utils.py)

In [ ]:
# Importing libraries
import my_utils as mu
import torch
from torch import nn

Task 1 Read dataset and create dataloaders

The default batch_size was 256, but after a number of tests using different values of batch_size such as 200, 128, 64, I found that 250 gives me the best test accuracy overall. 

train_iter, test_iter are the dataloaders loading fashion_mnist using the method predefined in my_utils.py.

In [ ]:
# Initialising batch size
batch_size = 250
# Data loader
train_iter, test_iter = mu.load_data_fashion_mnist(batch_size)

Task 2 Stem and Backbone

Stem

I used functional.unfold() instead of unfold().unfold() or reshape() in previous attempts as it is a better function to provide patching. 
Batch Normalisation is used to standardise inputs to linear layers which improves test accuracy.


Backbone

I used 2 MLPs, 2 ReLU activation softmax regression functions and 3 batch normalisations for my backbone.

In [ ]:
# Net class
class Net(torch.nn.Module):
    def __init__(self, num_inputs, num_hidden, num_outputs):
      # Constructor
      super(Net, self).__init__()
      # Initialise variables
      self.num_inputs = num_inputs
      self.num_hidden = num_hidden
      self.num_outputs = num_outputs

      # Stem Linear layer
      self.LinearStem = nn.Linear(49,49)
      # Stem linear layer normalisation
      torch.nn.init.normal_(self.LinearStem.weight, std=0.01)
      torch.nn.init.zeros_(self.LinearStem.bias)
      # Stem batch normalisation 1D
      self.batchNormStem = nn.BatchNorm1d(num_inputs)

      # Backbone Linear layers 
      # MLP 1
      self.LinearB1 = nn.Linear(num_inputs, num_hidden)
      # Backbone linear layer normalisation
      torch.nn.init.normal_(self.LinearB1.weight, std=0.01)
      torch.nn.init.zeros_(self.LinearB1.bias)
      # Backbone batch normalisation 1D
      self.batchNormBL1 = nn.BatchNorm1d(49)
      self.LinearB2 = nn.Linear(num_hidden,30)
      torch.nn.init.normal_(self.LinearB2.weight, std=0.01)
      torch.nn.init.zeros_(self.LinearB2.bias)
      self.batchNormBL2 = nn.BatchNorm1d(49)

      # MLP 2
      self.LinearB3 = nn.Linear(49, num_hidden)
      torch.nn.init.normal_(self.LinearB3.weight, std=0.01)
      torch.nn.init.zeros_(self.LinearB3.bias)
      self.batchNormBL3 = nn.BatchNorm1d(30)
      # No need for batch normalisation for final layer
      # Output from final layer gets passed into Classifier
      self.LinearB4 = nn.Linear(num_hidden,10)
      torch.nn.init.normal_(self.LinearB4.weight, std=0.01)
      torch.nn.init.zeros_(self.LinearB4.bias)

      # Activation function
      self.relu = nn.ReLU()


    def forward(self, image):
      # Stem
      # Unfold image
      imageUnfold = nn.functional.unfold(image, (4,4), stride=4)
      # Pass through linear layer and normalise batch
      out_stem = self.batchNormStem(self.LinearStem(imageUnfold))

      # Backbone
      # Block (each block uses 2 MLPs)
      # Transpose stem output
      out_backbone = torch.transpose(out_stem,1,2)
      # Pass through linear layer and normalise batch
      out_backbone = self.batchNormBL1(self.LinearB1(out_backbone))
      # Activate softmax regression
      out_backbone = self.relu(out_backbone)
      out_backbone = self.batchNormBL2(self.LinearB2(out_backbone))
      
      out_backbone = torch.transpose(out_backbone,1,2)
      out_backbone = self.batchNormBL3(self.LinearB3(out_backbone))
      out_backbone = self.relu(out_backbone)
      out_backbone = self.LinearB4(out_backbone)

      #  Pass to Classifier
      return out_backbone.mean(axis=1)

# Define input, hidden and output
num_inputs, num_hidden, num_outputs = 16, 250, 10

# Create Net class instance using defined values
net = Net(num_inputs, num_hidden, num_outputs)

Task 3 Loss % Optimizer

I used the default loss function nn.CrossEntropyLoss() that was given to us in the labs.

I changed the default optimizer SGD() to NAdam() as I have tried many test runs with SGD() and Adam() as well as NAdam(), NAdam() has given me the best test accuracy so far.

In [ ]:
# Loss
loss = nn.CrossEntropyLoss()

# Learning rate, Weight decay
lr, wd = 0.0009, 0.00001

# NAdam optimizer 
optimizer = torch.optim.NAdam(net.parameters(), lr=lr, weight_decay=wd)

In [ ]:
# Define time epoch
num_epochs = 30 
# Execute using predefined train_ch3() in my_untils.py
mu.train_ch3(net, train_iter, test_iter, loss, num_epochs, optimizer)
# mu.train_ch6(net, train_iter, test_iter, num_epochs, optimizer)
# Testing Accuracy
mu.evaluate_accuracy(net, test_iter)